In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')#for visbrain to work in jupyterlab/notebook
from header import *
import visbrain
from visbrain import Brain, Colorbar
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr

In [ ]:
task = 'SMEG'
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[0]
noise_cov = 'baseline_cov'
start = 0.35
stop = 0.5
sfreq = 200
path = op.join(Analysis_path, task, 'meg', 'Stats', str(start)+'_'+str(stop))
filename = op.join(path, name+'-'+noise_cov+'.nc')
test_key = 'RS_vs_OM+N'
p_thresh = 0.1

In [ ]:
stats = xr.open_dataarray(filename, test_key)
stats.load()

In [ ]:
if np.where(stats.loc['p_val', 'left']<p_thresh)[0].any():
    t_signif_lh = np.unique(np.where(stats.loc['p_val', 'left']<p_thresh)[0], return_counts=True)
    best_t_lh = stats.time[t_signif_lh[0][t_signif_lh[1].argsort()][::-1]]
    print(t_signif_lh)
    print(best_t_lh)
else:
    warnings.warn('\nNo significant cluster for left hemisphere.\n')
    best_t_lh = [None]

In [ ]:
if np.where(stats.loc['p_val', 'right']<p_thresh)[0].any():
    t_signif_rh = np.unique(np.where(stats.loc['p_val', 'right']<p_thresh)[0], return_counts=True)
    best_t_rh = stats.time[t_signif_rh[0][t_signif_rh[1].argsort()][::-1]]
    print(t_signif_rh)
    print(best_t_rh)
else:
    warnings.warn('\nNo significant cluster for right hemisphere.\n')
    best_t_rh = [None]

In [ ]:
t = float(best_t_rh[0] if best_t_rh[0] is not None else best_t_lh[0])
#t = int(best_t_rh[8])
#t = int(best_t_lh[0])
print('t =', t, 's')
mask_lh = np.where(stats.loc['p_val', 'left', t] < p_thresh)
mask_rh = np.where(stats.loc['p_val', 'right', t] < p_thresh)
lh = np.where(stats.loc['p_val', 'left', t] < p_thresh, stats.loc['T_stat', 'left', t].values, 0)
rh = np.where(stats.loc['p_val', 'right', t] < p_thresh, stats.loc['T_stat', 'right', t].values, 0)

In [ ]:
cmax = np.max(np.abs(np.concatenate([lh, rh])))
clim = (-cmax, cmax)

In [ ]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both')#, sulcus=True)

#b_obj.add_activation(data=data_lh[t], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_rh[t], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_lh[t]), hide_under=clim[0], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_rh[t]), hide_under=clim[0], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
b_obj.add_activation(data=lh, vertices=stats.src.values, hemisphere='left', smoothing_steps=4, cmap='bwr', clim=clim)
b_obj.add_activation(data=rh, vertices=stats.src.values, hemisphere='right', smoothing_steps=4, cmap='bwr', clim=clim)


vb = Brain(brain_obj=b_obj)
#cb = Colorbar(vmin=clim[0], vmax=clim[-1], cmap='bwr', ndigits=4, cblabel='T stat')

In [ ]:
vb.show()
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, t))
vb.rotate(custom=(90, 0))
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,'{}_t_{}.png'.format(test_key, t)))

In [ ]:
t_signif_rh